In [34]:
import requests
import tarfile
import os
from PIL import Image
import time
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import copy
import matplotlib.pyplot as plt

In [ ]:
## download stanford dog dataset
url = "http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar"
r = requests.get(url, allow_redirects = True) 
open("images.tar", "wb").write(r.content)

793579520

In [ ]:
### untaring the dataset
tar = tarfile.open("images.tar")
tar.extractall("./")
tar.close()

In [35]:
### check for gpus and assign to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
### image loader function and transform object

imageSize = (512, 512) if torch.cuda.is_available() else (128, 128)

loader = transforms.Compose([transforms.Resize(imageSize), 
                             transforms.ToTensor(),
                             transforms.Normalize((0.485, 0.456, 0.406),
                                                  (0.229, 0.224, 0.225))
                             ])

def imageLoader(imagePath):
  image = Image.open(imagePath)
  image = loader(image).unsqueeze(0)
  return image.to(device, torch.float)


In [37]:
samplePath = './Images/n02085620-Chihuahua/n02085620_10074.jpg'

sampleImage = imageLoader(samplePath)

print(type(sampleImage))
print(sampleImage.size())

<class 'torch.Tensor'>
torch.Size([1, 3, 128, 128])


In [38]:
### variable initialization
dataDir = "./Images"
testSplit = 0.2
epochs = 10
nClasses = 120
featureExtract = True
modelName = "resnet"

In [39]:
### load data and split into train and test
trainLabels = []
trainImages = []
mapping = []
testLabels = []
testImages = []

sampleList = ['./Images/n02108000-EntleBucher', './Images/n02111889-Samoyed']

for i, (root, dirs, files) in enumerate(os.walk(dataDir)):
  if root != dataDir and root in sampleList:    ### add root in sampleList condition only for testing
    name = root.split("/")[-1].split("-")[-1]
    mapping.append(name)
    print(root)
    print(name)
    nFiles = len(files)
    for j, f in enumerate(files):
      filePath = os.path.join(root, f)
      image = imageLoader(filePath)
      if j < (nFiles * (1 - testSplit)):
        trainImages.append(image)
        trainLabels.append(i-1)
      else:
        testImages.append(image)
        testLabels.append(i-1)

trainData = {}
trainData["trainImages"] = trainImages
trainData["trainLabels"] = trainLabels
    
print(trainData.keys())

./Images/n02108000-EntleBucher
EntleBucher
./Images/n02111889-Samoyed
Samoyed
dict_keys(['trainImages', 'trainLabels'])


In [40]:
### set requires_grad for model parameters

def setParameterRequiresGrad(model, featureExract):
  if featureExtract:
    for param in model.parameters():
      param.requires_grad = False

In [41]:
### model initialization

def initializeModel(modelName, nClasses, featureExtract, 
                    use_pretrained = True):
  modelFt = None
  inputSize = 0

  if modelName == "resnet":
    modelFt = models.resnet50(pretrained = use_pretrained)
    setParameterRequiresGrad(modelFt, featureExtract)
    nFeatures = modelFt.fc.in_features
    modelFt.fc = nn.Linear(nFeatures, nClasses)
    inputSize = 224

  else:
    print("Invalid model name, exiting...")
    exit()

  return modelFt, inputSize 

modelFt, inputSize = initializeModel(modelName, nClasses,
                                     featureExtract)
print(modelFt)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [42]:
### model GPU initialization, optimizer initialization

## send model to GPU
modelFt = modelFt.to(device)


## print parameters to learn
print("Parameters to learn")
if featureExtract:
  paramsUpdate = []
  for name, param in modelFt.named_parameters():
    if param.requires_grad == True:
      paramsUpdate.append(param)
      print("\t", name)
else:
  paramsUpdate = modelFt.parameters()
  for name, param in modelFt.named_parameters():
    if param.requires_grad == True:
      print("\t", name)


## initialize optimizer
optimizerFt = optim.SGD(paramsUpdate, lr = 0.001, momentum=0.9) 

Parameters to learn
	 fc.weight
	 fc.bias


In [55]:
### model training function

def trainModel(model, trainData, criterion, optimizer,
               nEpochs = 10):
  start = time.time()

  accHistory = []

  bestModelWts = copy.deepcopy(model.state_dict())
  bestAcc = 0

  for epoch in range(nEpochs):
    print(f'Epoch {epoch}/{nEpochs - 1}')
    print('-' * 10)

    runningLoss = 0.0
    runningCorrects = 0

    inputs = trainData["trainImages"]
    labels = trainData["trainLabels"]

    for i, input in enumerate(inputs):
      input = input.to(device)
      label = np.empty([1], dtype = np.int64)
      np.insert(label, 0, labels[i])
      label = torch.tensor(label).to(device)

      ##zero gradients
      optimizer.zero_grad()

      output = model(input)
      loss = criterion(output, label)

      _, prediction = torch.max(output, 1)

      loss.backward()
      optimizer.step()

      runningLoss += loss.item() * input.size(0)
      runningCorrects += torch.sum(prediction == label)

    epochLoss = runningLoss / (len(inputs))
    epochAcc = runningCorrects.double() / (len(inputs))

    if epochAcc > bestAcc:
      bestAcc = epochAcc
      bestModelWts = copy.deepcopy(model.state_dict())

    accHistory.append(epochAcc)

  timeTaken = time.time() - start
  print("training complete in {:.0f}m {:.0f}s".format(timeTaken // 60,
                                         timeTaken % 60))
  
  print("Best training accuracy: {:4f}".format(bestAcc))

  model.load_state_dict(bestModelWts)

  return model, accHistory




In [56]:
criterion = nn.CrossEntropyLoss()

modelFt , hist = trainModel(modelFt, trainData, criterion,
                            optimizerFt, nEpochs = epochs)

Epoch 0/9
----------
Epoch 1/9
----------
Epoch 2/9
----------
Epoch 3/9
----------
Epoch 4/9
----------
Epoch 5/9
----------
Epoch 6/9
----------
Epoch 7/9
----------
Epoch 8/9
----------
Epoch 9/9
----------
training complete in 11m 5s
Best training accuracy: 1.000000


In [ ]:
sampleDict = {
    "a" : 10,
    "b" : 20,
    "c" : 30
}

for a, b, c in sampleDict.keys():
  print(a, b, c)

dict_keys(['a', 'b', 'c'])


In [ ]:
# print(imag)
# print(mapping)
# print(labels)
# print(images)
print(len(mapping))
print(len(trainLabels))
print(len(trainImages))
print(len(testLabels))
print(len(testImages))
print(trainImages[0].size())
print(testImages[0].size())
print(model.children)

6
337
337
83
83
torch.Size([1, 3, 128, 128])
torch.Size([1, 3, 128, 128])
<bound method Module.children of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU

NameError: ignored

1604319532.5352366
